# CSV Manipulation

Let's read, calculate, and modify a CSV in-place.

## CSV

First, we'll need a CSV file to work with. How about a list of spheres? Most importantly, note the radius in the third column:

In [1]:
input_csv = ['first,second,radius',
            'this,is,1',
            'this,is,2',
            'this,is,3']

This as a list of lines as you would read from a CSV file. We are treating this like a real CSV file. However, if you were instead doing this with a file on disk, you'd read the file in like so:

```python
# imports

# functions

with open('spheres.csv', 'r+') as input_csv:
    # Calculation code here.
    
    # Output back to the same file we read
    # instead of printing the output.
    csv.writer(input_csv, delimiter=',').writerows(output_csv)
```

## Imports

None of these are strictly needed, but provide some additional clarity and accuracy.

- The `csv` module provides helpers for working with CSV data.
- The `math` module will provide:
    - The constant `pi`
    - A function for power/exponent `pow()`.

In [2]:
import csv
from math import pow,pi

## Functions

There are a few functions that we'll want to help with our calculations. Though they are not particularly long functions, they will help to make the rest of the code easier to read.

One for calculating diameter,

In [ ]:
def diameter(rad):
    return rad*2

one for surface area,

In [ ]:
def surface(rad):
    return 4*pi*pow(rad,2)

and one for volume.

In [3]:
def volume(rad):
    return (4/3)*(pi*pow(rad,3))

## Read and Manipulate

We use the `csv.reader()` function to read the CSV data.

> This is a little overkill for our needs since we have three lines of data, but if you had to do something more complex, such as working with Excel's dialect, then this would help. The `pandas` module woudl be appropriate for larger datasets and calculations.

In [4]:
csv_reader = csv.reader(input_csv, delimiter=',')

We need to detect whether we're reading a header or a line of data. All of our calculations will error out if he handed them column titles instead of numbers.

> You might also do this using a `try`/`catch` around `radius = float(row[2])` looking for `ValueError`. Some folks will count lines. It's really up to the author and the performance constraints.

Setting `header` to `True` will be an easy piece of _state_ that we can manage ourselves. Since the header is the first line, we'll just set `header` to `False` after the first iteration through the loop.

In [5]:
header = True

Create an empty list of rows to store the data from the CSV _file_ and our calculations. Each item in the list representing a line in a CSV file.

In [6]:
output_csv = []

Let's _loop_ through each line in our current CSV data, decide if we're at the header line or actual data, calculate, and append our new data to each line.

In [7]:
for row in csv_reader:
    # Are we at the header row?
    if header:
        # We are! Add new headers.
        output_csv.append(row + ["diameter","surface","volume"])
        # Set header to False now that
        # the rest of the file is data.
        header = False
        # Skip the rest of the loop execution for
        # this iteration. Go to the next line of the file.
        continue
    # Find and assign the radius.
    radius = float(row[2])
    # Append our calculations to the end of each row.
    output_csv.append(row + [diameter(radius),surface(radius),volume(radius)])

Finally, let's see the output we end up with.

In [8]:
for row in output_csv:
    print(row)

Plenty of room for refactoring (e.g. list comprehensions, neither `matn.pow()` or `csv` are _strictly_ needed...), but this is a general demonstration.